In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import sys
sys.path.insert(0,'../')
import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+06f25f6 torch: 0.3.0.post4 cuda: True


In [2]:
from functools import wraps

# TODO: move this into probtorch.util
def expand_inputs(f):
    @wraps(f)
    def g(*args, num_samples=None, **kwargs):
        if not num_samples is None:
            new_args = []
            new_kwargs = {}
            for arg in args:
                if hasattr(arg, 'expand'):
                    new_args.append(arg.expand(num_samples, *arg.size()))
                else:
                    new_args.append(arg)
            for k in kwargs:
                arg = kwargs[k]
                if hasattr(arg, 'expand'):
                    new_args.append(arg.expand(num_samples, *arg.size()))
                else:
                    new_args.append(arg)
            return f(*new_args, num_samples=num_samples, **new_kwargs)
        else:
            return f(*args, num_samples=None, **kwargs)
    return g

# Parameters

In [3]:
# Model Paramters:
NUM_LATENT = 64

# Training Parameters:
NUM_SAMPLES = 1
NUM_BATCH = 1024
NUM_EPOCHS = 100
LABEL_FRACTION = 0.0
LEARNING_RATE = 1e-3
EPS = 1e-9
CUDA = torch.cuda.is_available()

# LOSS parameters:
ALPHA = 0.1
BETA = (4.0, 1.0, 0.0, 4.0, 1.0) 

# Encoder Model

In [4]:
class Encoder(nn.Module):
    def __init__(self, num_latent=NUM_LATENT):
        super(self.__class__, self).__init__()
        self.enc = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1),     # 32 x 32
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1),   # 16 x 16
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 4, 2, 1),  # 8 x 8
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1),  # 4 x 4
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 512, 4),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            # nn.Conv2d(512, num_latent, 1),
        )
        self.mu = nn.Conv2d(512, num_latent, 1)
        self.logvar = nn.Conv2d(512, num_latent, 1)

    @expand_inputs
    def forward(self, images, labels=None, num_samples=None):
        q = probtorch.Trace()
        enc = self.enc(images.view(-1, 3, 64, 64))
        mu = self.mu(enc).squeeze()
        logvar = self.logvar(enc).squeeze()
        latent = q.normal(mu, logvar, name='z')  # need to figure out how they set stdev
        return q 

# Decoder Model

In [5]:
def binary_cross_entropy(x_mean, x, EPS=1e-9):
    return - (torch.log(x_mean + EPS) * x + 
              torch.log(1 - x_mean + EPS) * (1 - x)).sum(-1)

class Decoder(nn.Module):
    def __init__(self, num_latent=NUM_LATENT):        
        super(self.__class__, self).__init__()
        self.dec = nn.Sequential(
            nn.ConvTranspose2d(num_latent, 512, 1, 1, 0),  # 1 x 1
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 256, 4, 1, 0),         # 4 x 4
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),         # 8 x 8
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 128, 4, 2, 1),         # 16 x 16
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),          # 32 x 32
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1)             # 64 x 64
        )
        
        
    def forward(self, images, q=None, num_samples=None):
        p = probtorch.Trace()
        b_size = images.size(0)
        latent = p.normal(0.0, 1.0, value=q['z'], name='z')
        images_mean = self.dec(latent.view(b_size, -1, 1, 1)).view(b_size, 3, 64, 64).sigmoid()
        p.loss(binary_cross_entropy, images_mean, images, name='images')
        return p

# Loss

In [6]:
def elbo(q, p, alpha=ALPHA, beta=BETA):
    return probtorch.objectives.montecarlo.elbo(q, p, alpha=alpha, beta=beta[0])

# Data

In [7]:
from torchvision import datasets, transforms
import os 
DATA_PATH = '../data/celeb-a/resized_img_align_celeba'

if not os.path.isdir(DATA_PATH):
    os.makedirs(DATA_PATH)

class CelebADataset(datasets.ImageFolder):
    """Subclass of ImageFolder specific to celebA"""
    def __init__(self, root, split='train', transform=None):
        super(CelebADataset, self).__init__(root + '/' + split, transform=transform)
        self.attributes = np.load(root + '/' + split + '/attributes.npy')[()]

    def __getitem__(self, index):
        img, _ = super(CelebADataset, self).__getitem__(index)
        img_name = os.path.basename(self.imgs[index][0])
        attr = self.attributes[img_name]
        return img, torch.Tensor(attr)

train_data = torch.utils.data.DataLoader(
                CelebADataset(DATA_PATH, split='train', transform=transforms.ToTensor()),
                batch_size=NUM_BATCH, shuffle=True)
test_data = torch.utils.data.DataLoader(
                CelebADataset(DATA_PATH, split='validation', transform=transforms.ToTensor()),
                batch_size=NUM_BATCH, shuffle=True) 

# Training and Testing

In [8]:
enc = Encoder()
dec = Decoder()
if CUDA:
    enc.cuda()
    dec.cuda()
optimizer =  torch.optim.Adam(list(enc.parameters())+list(dec.parameters()),
                              lr=LEARNING_RATE)


In [10]:
def train(data, enc, dec, optimizer, 
          label_mask={}, label_fraction=LABEL_FRACTION):
    epoch_elbo = 0.0
    enc.train()
    dec.train()
    for b, (images, labels) in enumerate(data):        
        #images = images.view(-1, NUM_PIXELS)
        if CUDA:
            images = images.cuda()
        images = Variable(images)
        optimizer.zero_grad()
        if b not in label_mask:
            label_mask[b] = (random() < label_fraction)
        if label_mask[b]:
            labels_onehot = torch.zeros(NUM_BATCH, 1)
            #labels_onehot.scatter_(1, labels.unsqueeze(1), 1)
            #labels_onehot = torch.clamp(labels_onehot, EPS, 1-EPS)
            #labels_onehot = Variable(labels_onehot)
            if CUDA:
                labels_onehot = labels_onehot.cuda()
            q = enc(images, labels_onehot, num_samples=NUM_SAMPLES)
        else:
            q = enc(images, num_samples=NUM_SAMPLES)
        p = dec(images, q, num_samples=NUM_SAMPLES)
        loss = -elbo(q, p)
        loss.backward()
        optimizer.step()
        if CUDA:
            loss = loss.cpu()
        epoch_elbo -= loss.data.numpy()[0]
    return epoch_elbo / len(train_data.dataset), label_mask


def test(data, enc, dec, infer=True):
    enc.eval()
    dec.eval()
    epoch_elbo = 0.0
    epoch_correct = 0
    N = len(test_data.dataset)
    for b, (images, labels) in enumerate(data):
        if CUDA:
            images = images.cuda()
        images = Variable(images)
        q = enc(images, num_samples=NUM_SAMPLES)
        p = dec(images, q, num_samples=NUM_SAMPLES)
        batch_elbo = elbo(q, p)
        if CUDA:
            batch_elbo = batch_elbo.cpu()
        epoch_elbo += batch_elbo.data.numpy()[0]

    return epoch_elbo / N, epoch_correct / N


In [ ]:
import time
from random import random

train_elbo_dict = {'ELBO':[], 'Recun Loss': [], 'TC': [], 'PriorZ':[], 'PriorY':[], 'MI':[]}
test_elbo_dict = {'ELBO':[], 'Recun Loss': [], 'TC': [], 'PriorZ':[], 'PriorY':[], 'MI':[]}
test_acc_list = []
mask = {}
for e in range(NUM_EPOCHS):
    train_start = time.time()
    train_elbo, mask = train(train_data, enc, dec, 
                             optimizer, mask, LABEL_FRACTION)
    train_end = time.time()
    test_start = time.time()
    test_elbo, test_accuracy = test(test_data, enc, dec)
    test_end = time.time()
    
    test_acc_list.append(test_accuracy)
    print('[Epoch %d] Train: ELBO %.4e (%ds) Test: ELBO %.4e, Accuracy %0.3f (%ds)' % (
            e, train_elbo, train_end - train_start, 
            test_elbo, test_accuracy, test_end - test_start))

[Epoch 0] Train: ELBO -7.5535e+03 (85s) Test: ELBO -7.3293e+03, Accuracy 0.000 (7s)
[Epoch 1] Train: ELBO -7.2447e+03 (84s) Test: ELBO -7.2323e+03, Accuracy 0.000 (7s)
[Epoch 2] Train: ELBO -7.2632e+03 (84s) Test: ELBO -7.4186e+03, Accuracy 0.000 (7s)
[Epoch 3] Train: ELBO -7.2160e+03 (84s) Test: ELBO -7.1839e+03, Accuracy 0.000 (7s)
[Epoch 4] Train: ELBO -7.1601e+03 (84s) Test: ELBO -7.2093e+03, Accuracy 0.000 (7s)
[Epoch 5] Train: ELBO -7.1757e+03 (84s) Test: ELBO -7.2307e+03, Accuracy 0.000 (7s)
[Epoch 6] Train: ELBO -7.1507e+03 (84s) Test: ELBO -7.1459e+03, Accuracy 0.000 (7s)
[Epoch 7] Train: ELBO -7.1807e+03 (84s) Test: ELBO -7.1719e+03, Accuracy 0.000 (7s)
[Epoch 8] Train: ELBO -7.3905e+03 (84s) Test: ELBO -7.2913e+03, Accuracy 0.000 (7s)
[Epoch 9] Train: ELBO -7.2394e+03 (84s) Test: ELBO -7.2258e+03, Accuracy 0.000 (7s)
[Epoch 10] Train: ELBO -7.2243e+03 (84s) Test: ELBO -7.2306e+03, Accuracy 0.000 (7s)
[Epoch 11] Train: ELBO -7.2071e+03 (84s) Test: ELBO -7.2270e+03, Accuracy 0

# Visualise Reconstructions

In [ ]:
from torchvision.utils import make_grid

def show(img):
    npimg = img.numpy()
    plt.figure(figsize=(20, 2))
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    plt.tight_layout()

test_DL = torch.utils.data.DataLoader(
            CelebADataset(DATA_PATH, split='validation', transform=transforms.ToTensor()),
            batch_size=15, shuffle=True) 

In [ ]:
for i, (images, _) in enumerate(test_DL):
    if CUDA:
        images = images.cuda()
    images = Variable(images)
    q = enc(images, num_samples=NUM_SAMPLES)
    p = dec(images, q, num_samples=NUM_SAMPLES)
    recon = p['images'].value
    break

show(make_grid(torch.cat([images, recon], 0).data.cpu(), nrow=15))

# Visualize Encodings

In [ ]:
# Get all the embeddings
Zs = []
for b, (images, labels) in enumerate(train_data):
    enc.eval()
    if CUDA:
        images = images.cuda()
    images = Variable(images, volatile=True)
    q = enc(images, num_samples=NUM_SAMPLES)
    Zs.append(q['z'].value)

Zs = torch.cat(Zs, 0).data.squeeze().cpu().numpy()

In [ ]:
f, axarr = plt.subplots(NUM_LATENT, NUM_LATENT, figsize=(30, 30), sharex=True)
f.suptitle(r'$Z \ Embeddings$' , fontsize=30)

for i in range(NUM_LATENT):
    axarr[NUM_LATENT-1,i].set_xlabel(r'$\mathbf{z_{%d}}$' % i, fontsize=10)
    axarr[i,0].set_ylabel(r'$\mathbf{z_{%d}}$' % i, fontsize=10)
    for j in range(NUM_LATENT):
        if i==j:
            axarr[j,i].hist(Zs[:,i], bins=40)
        else:
            axarr[j,i].scatter(Zs[:,i],Zs[:,j],alpha=0.5)
            
plt.show()

# Go no further!!

# Let's see some Features!

In [ ]:
def vary_z2(index, zmin, zmax):
    f, axarr = plt.subplots(10,10,figsize=(10,10),sharey=True)
    f.suptitle(r'$\mathbf{z_{%d}} \ \  varying$' % index , fontsize=30)
    z_range = np.linspace(zmin,zmax,num=10)
    
    for i in range(10):
        for j in range(10):
            z = torch.zeros(1,10)
            z[0,index] = z_range[i]
            z = Variable(z.cuda())
            null_image = Variable(torch.Tensor(np.zeros((1, 784)))).cuda()
            y_hot = np.zeros((1,10))
            y_hot[0,j] = 1
            y_hot = Variable(torch.FloatTensor(y_hot).cuda())
            q_null = {'z': z, 'y':y_hot}
            p = dec(null_image, q_null, num_samples=NUM_SAMPLES) 
            image = p['images']
            image = image.value.cpu().data.numpy().reshape(28,28)
            axarr[i,j].imshow(image)
            axarr[i,j].axis('off')
            
    return None

for style in range(NUM_STYLE):
    vary_z2(style, -3, 3)
    plt.show()

1. feature 0: Horizontal angles (look at 3 ,4, and 7)
2. feature 1: Y-position of the middle line (look at 4 middle line)
3. feature 2: Width
4. feature 3: Nothing
5. feature 4: Angle 
6. feature 5: Nothing
7. feature 6: Stroke
8. feature 7: Not sure
9. feature 8: Top circle gets bigger, the other one smaller
10. feature 9: Nothing

## Features $z_{i}$ vs $z_{j}$ for Digits:

In [ ]:
def zi_vs_zj(z_index1, z_index2, zmin=3, zmax=3, num_z=10, digit=0):
    f, axarr = plt.subplots(num_z, num_z, figsize=(num_z, num_z), sharey=True)
    f.suptitle(r'$Digit: %s$' % digit , fontsize=30)
    z_range = np.linspace(zmin,zmax,num=num_z)
    
    for i in range(num_z):
        for j in range(num_z):
            z = torch.zeros(1,NUM_STYLE)
            z[0,z_index1] = z_range[i]
            z[0,z_index2] = z_range[j]
            z = Variable(z.cuda())
            y_hot = np.zeros((1,NUM_DIGITS))
            y_hot[0,digit] = 1
            y_hot = Variable(torch.FloatTensor(y_hot).cuda())
            q_null = {'z':z, 'y':y_hot}
            null_image = Variable(torch.Tensor(np.zeros((1, NUM_PIXELS)))).cuda()
            p = dec(null_image, q_null, num_samples=NUM_SAMPLES) 
            image = p['images']
            pixels = int(np.sqrt(NUM_PIXELS))
            image = image.value.cpu().data.numpy().reshape(pixels,pixels)
            axarr[i,j].imshow(image)
            axarr[i,j].axis('off')
    f.text(0.52, 0.08, r'$\mathbf{z_{%d}}$' % z_index2, ha='center', fontsize=20)
    f.text(0.09, 0.5, r'$\mathbf{z_{%d}}$' % z_index1, va='center', rotation='vertical', fontsize=20)

In [ ]:
z_index1 = 6
z_index2 = 4
for digit in range(NUM_DIGITS):
    zi_vs_zj(z_index1, z_index2, zmin=-3, zmax=3, num_z=10, digit=digit)
    plt.show()

## Track terms Vs iters

In [ ]:
fig = plt.figure(figsize=(20,10))
iters = np.arange(NUM_EPOCHS)
for label in train_elbo_dict:
    plt.plot(iters, train_elbo_dict[label], label=label, linewidth=3)
plt.legend(prop={'size': 20})
plt.xlabel("iter", fontsize=20)
plt.title(" Training Process", fontsize=20)
plt.grid(True)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
iters = np.arange(NUM_EPOCHS)
for label in test_elbo_dict:
    plt.plot(iters, test_elbo_dict[label], label=label, linewidth=3)
plt.legend(prop={'size': 20})
plt.xlabel("iter", fontsize=20)
plt.title(" Testing Process", fontsize=20)
plt.grid(True)
plt.show()